In [1]:
import repeatrand as rr
import unittest

BOARD_X = 4
BOARD_Y = 4
BOARD_EDGES = BOARD_X * BOARD_Y * 2

WHITE = 'W'
BLACK = 'B'

HORIZ = 1
VERT = 2


rand = rr.repeatable_random(rr.rand_seed())

# numbering of edges:
#   sides are 0-9,0-9
#   squares are X
#
#   . 0 . 2 . 4 . 6
#   1 X 3 X 5 X 7 
#   . 8 . 0 . 2 . 4
#   9 X 1 X 3 X 5
#   . 6 . 8 . 0 . 2
#   7 X 9 X 1 X 3
#   . 4 . 6 . 8 . 
#
"""
Alternate way to think about this:
    N x N squares
    Each square has four sides
    Some sides may be a side of another square too
    So edges[square][side]
    and a mapping of also_edge[square][side]
"""

class Board(object):
    def __init__(self):
        self.xdim = BOARD_X
        self.ydim = BOARD_Y
        
        self.b = [False for x in range(self.xdim * self.ydim * 2)]
        self.pip = [0 for x in range(self.xdim * self.ydim)]
    
        self.valid_moves = self.setup_valid_moves()
    
    def print(self):
        o = 0
        res = ""
        for y in range(self.ydim):
            l1 = ""
            l2 = ""
            for x in range(self.xdim):
                if self.b[o]:
                    l1 += "*---"
                else:
                    l1 += "*   "
                if not (self.b[o+1]):
                    l2 += "    "
                elif (self.pip[int(o/2)] != 0):
                    l2 += "| {} ".format(self.pip[int(o/2)])
                else:
                    l2 += "|   "
                o += 2
            res += l1 + "\n"
            res += l2 + "\n"
        return res

    def setup_valid_moves(self):
        v = [x for x in range(2 * BOARD_X * BOARD_Y)]

        x = self.xdim - 1
        for y in range(self.ydim):
            v.remove((y * self.xdim + x) * 2)

        y = self.ydim - 1
        for x in range(self.xdim):
            v.remove((y * self.xdim + x) * 2 + 1)
        
        return v

    # See if this completed a square. 
    #   m == move played
    def check_win(self, m):
        if (m < 0):
            return
        
        m &= ~1
        b = self.b
        return b[m] and b[m + 1] and b[m + 3] and b[m + self.xdim*2]

    def mark_win(self, ind, who):
        if (who != BLACK) and (who != WHITE):
            raise ValueError("who is bogus " + str(who))

        if (self.pip[int(ind/2)] == 0):
            #print("pip {} == {}".format(int(ind/2), who))
            self.pip[int(ind/2)] = who
            #print("self.pip=" + str(self.pip))
            return True
        return False

    def move(self, who, ind):
        if (who != BLACK) and (who != WHITE):
            raise ValueError("who is bogus " + str(who))

        x = int(ind / 2 % self.xdim)
        y = int(ind / 2 / self.xdim)
        
        if (not ind in self.valid_moves):
            raise ValueError("{} is not a valid move".format(ind))
        if (x < 0) or (x >= self.xdim):
            raise ValueError("x out of bounds: " + str(x))
        if (y < 0) or (y >= self.ydim):
            raise ValueError("y out of bounds: " + str(y))
        if (x == self.xdim - 1) and (ind & 1 == 0):
            raise ValueError("cannot draw horizontal on last item")
        if (y == self.ydim - 1) and (ind & 1):
            raise ValueError("cannot draw vertical on last item")
        if self.b[ind]:
            raise ValueError("cell is already set " + str(ind))

        self.valid_moves.remove(ind)
        self.b[ind] = True
        
        # TODO pip array is messed up
        w1 = w2 = w3 = False
        if self.check_win(ind):
            w1 = self.mark_win(ind, who)
        if self.check_win(ind - 2):
            w2 = self.mark_win(ind - 2, who)
        if self.check_win(ind - self.xdim * 2):
            w3 = self.mark_win(ind - self.xdim * 2, who)
        
        return w1 or w2 or w3

    def neighbors(self, x, y):
        b = self.b

        res = 0
        base = x * 2 + y * self.xdim * 2
        
        #print("for {},{} coord are {},{},{},{}".format(x, y, base, base + 1, base + 3, base + self.xdim * 2))
        if b[base]:
            res |= 1
        if b[base + 1]:
            res |= 2
        if b[base + 3]:
            res |= 4
        if b[base + self.xdim * 2]:
            res |= 8
        return res

    def missing_neighbor(self, x, y):
        b = self.b

        res = 0
        base = x * 2 + y * self.xdim * 2
        
        #print("for {},{} coord are {},{},{},{}".format(x, y, base, base + 1, base + 3, base + self.xdim * 2))
        for z in [base, base + 1, base + 3, base + self.xdim * 2]:
            if not b[z]:
                return z
        
        raise ValueError("Unable to find missing neighbor {} {}".format(x, y))

    def final_score(self):
        x = sum([1 if i == WHITE else 0 for i in self.pip])
        y = sum([1 if i == BLACK else 0 for i in self.pip])
        return x, y

    def winner(self):
        x, y = self.final_score()
        if (x > y):
            return WHITE
        else:
            return BLACK

In [2]:


class Player(object):
    def __init__(self, name, symbol):
        if (symbol != BLACK) and (symbol != WHITE):
            raise ValueError("symbol must be BLACK or WHITE " + str(symbol))

        self.name = name
        self.symbol = symbol
    
    def new_game(self):
        self.name = self.name
        
    def select_move(self, board):
         raise NotImplementedError
    
    def finalResult(self, winner):
        pass

# plays a random move out of the set of legal moves
class RandomPlayer(Player):
    def __init__(self, name, symbol):
        super().__init__(name, symbol)
        
    def select_move(self, board):
        m = board.valid_moves[rand.rand_int(len(board.valid_moves))]
        return m

# takes a square if it can without regard for long-term
class GreedyPlayer(RandomPlayer):
    def __init__(self, name, symbol):
        super().__init__(name, symbol)
        
    def bits_set(self, val):
        return bin(val).count("1")

    # walk each square. if it has three lines already, return it. Otherwise pick a random move.
    def select_move(self, b):
        for x in range(b.xdim - 1):
            for y in range(b.ydim - 1):
                #print("move {},{} bits {}".format(x, y, bits_set(b.neighbors(x, y))))
                neighbors = b.neighbors(x, y)
                if self.bits_set(neighbors) == 3:
                    #print("greedy wants {},{}".format(x,y))
                    m = b.missing_neighbor(x, y)
                    #print("greedy play is {}".format(m))
                    #print(b.print())
                    return m
        return super().select_move(b)
    
# A single game, P1 white vs P2 black
class Game(object):
    def __init__(self, p1, p2):
        p1.next_player = p2
        p2.next_player = p1
        self.to_play = p1
        self.board = Board()
    
    def current_player(self):
        return self.to_play;
    
    def next_player(self):
        self.to_play = self.to_play.next_player
        return self.current_player()

    # plays the game out from the current position, returns the winning symbol.
    def play_game(self):
        #print("starting game {} vs {}".format(self.to_play.symbol, self.to_play.next_player.symbol))
        
        b = self.board
        self.to_play.new_game()
        self.to_play.next_player.new_game()
        
        player = self.current_player()
        while (len(b.valid_moves) > 0):
            #print(b.print())
            move = player.select_move(b)
            #print("{} selects {}".format(self.current_player().symbol, move))
            
            if not b.move(player.symbol, move):
                player = self.next_player()
        winner = b.winner()
        self.to_play.finalResult(self.to_play.symbol == winner)
        self.to_play.next_player.finalResult(self.to_play.next_player.symbol == winner)
        #print("b.winner=" + str(b.winner()))
        return b.winner()

# play a set of matches, each consisting of games_per_match games.
class Series(object):
    def __init__(self, p0, p1, games_per_match, matches):
        self.players = [p0, p1]
        self.p0 = p0
        self.p1 = p1
        self.toPlay = 0
        self.board = Board()
        self.matches = matches
        self.update_frequency = max(matches / 20, 1)
        self.games_per_match = games_per_match
    
    def printResults(self, matches_for_0, matches_for_1):
        print("{} won {} ({:.0%}) matches, {} won {} ({:.0%})".format(
            self.p0.name, matches_for_0, matches_for_0/(matches_for_0+matches_for_1+0.01),
            self.p1.name, matches_for_1, matches_for_1/(matches_for_0+matches_for_1+0.01)))
        
    def run(self, wins0, wins1):
        matches_for_0 = 0
        matches_for_1 = 0
        wins0.append(matches_for_0)
        wins1.append(matches_for_1)
        for matchno in range(1, self.matches + 1):
            wins_for_0 = 0
            for _ in range(self.games_per_match):
                g = Game(self.p0, self.p1)
                winner = g.play_game()
                #print("{} won".format(winner))
                if (winner == self.p0.symbol):
                    wins_for_0 += 1
                #print(g.board.print())
            if (wins_for_0 > self.games_per_match - wins_for_0):
                matches_for_0 += 1
            else:
                matches_for_1 += 1
            if matchno % self.update_frequency == 0:
                self.printResults(matches_for_0, matches_for_1)
                wins0.append(matches_for_0/matchno)
                wins1.append(matches_for_1/matchno)
        
        wins0.append(matches_for_0/matchno)
        wins1.append(matches_for_1/matchno)
        self.printResults(matches_for_0, matches_for_1)

In [8]:
p0 = GreedyPlayer("Greedy", WHITE)
p1 = RandomPlayer("Random", BLACK)

m = Series(p0, p1, 3, 100)
#m.run([], [])

Greedy won 4 (100%) matches, Random won 0 (0%)
Greedy won 8 (100%) matches, Random won 0 (0%)
Greedy won 12 (100%) matches, Random won 0 (0%)
Greedy won 16 (100%) matches, Random won 0 (0%)
Greedy won 20 (100%) matches, Random won 0 (0%)
Greedy won 24 (100%) matches, Random won 0 (0%)
Greedy won 28 (100%) matches, Random won 0 (0%)
Greedy won 32 (100%) matches, Random won 0 (0%)
Greedy won 36 (100%) matches, Random won 0 (0%)
Greedy won 40 (100%) matches, Random won 0 (0%)
Greedy won 44 (100%) matches, Random won 0 (0%)
Greedy won 48 (100%) matches, Random won 0 (0%)
Greedy won 52 (100%) matches, Random won 0 (0%)
Greedy won 56 (100%) matches, Random won 0 (0%)
Greedy won 60 (100%) matches, Random won 0 (0%)
Greedy won 64 (100%) matches, Random won 0 (0%)
Greedy won 68 (100%) matches, Random won 0 (0%)
Greedy won 72 (100%) matches, Random won 0 (0%)
Greedy won 76 (100%) matches, Random won 0 (0%)
Greedy won 80 (100%) matches, Random won 0 (0%)
Greedy won 84 (100%) matches, Random won 0

In [91]:
TOP = 1
LEFT = 2
BOTTOM = 4
RIGHT = 8
directions = { TOP: "top", BOTTOM: "bottom", LEFT: "left", RIGHT: "right"}

def encode(x, y, d):
    z = (y * BOARD_Y + x) * 16 + d
    #print("{}, {}, {} = {}".format(x, y, d, z))
    return z

class NewBoard(object):
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.lines =  [[0] * BOARD_Y for i in range(BOARD_X)]
        self.mapped = [None] * BOARD_X * BOARD_Y * 16
        
        for i in range(x - 1):
            for j in range(y):
                self.mapped[encode(i, j, RIGHT)] = (i + 1, j, LEFT)
        for i in range(1, x):
            for j in range(y):
                self.mapped[encode(i, j, LEFT)] = (i - 1, j, RIGHT)
        for i in range(x):
            for j in range(y - 1):
                self.mapped[encode(i, j, BOTTOM)] = (i + 1, j, TOP)
        for i in range(x):
            for j in range(1, y):
                self.mapped[encode(i, j, TOP)] = (i - 1, j, BOTTOM)

    def alreadyPlayed(self, x, y, d):
        return (self.lines[y][x] & d) != 0
    
    def move(self, x, y, d):
        if (self.alreadyPlayed(x, y, d)):
            raise ValueError("{},{},{} has already been played".format(x, y, directions[d]))
        if (x < 0) or (x >= self.x):
            raise ValueError("x out of bounds: " + str(x))
        if (y < 0) or (y >= self.y):
            raise ValueError("y out of bounds: " + str(y))


        result = []
        
        self.lines[y][x] |= d
        if (self.lines[y][x] == LEFT + RIGHT + TOP + BOTTOM):
            result.append((x, y))
        
        e = self.mapped[encode(x, y, d)]
        #print(e)
        if not (e is None): 
            self.lines[e[0]][e[1]] |= e[2]
            if (self.lines[e[0]][e[1]] == LEFT + RIGHT + TOP + BOTTOM):
                result.append((e[0],e[1]))

        return result
    
class TestBoard(unittest.TestCase):
    def test_drawline_dup(self):
        try:
            b = NewBoard(BOARD_X, BOARD_Y)
            b.move(1, 1, TOP)
            b.move(1, 1, TOP)
        except ValueError:
            pass

    def test_drawline_top_bottom_conflict(self):
        try:
            b = NewBoard(BOARD_X, BOARD_Y)
            b.move(1, 1, BOTTOM)
            b.move(1, 2, TOP)
        except ValueError:
            pass

    def test_mapping1(self):
        b = NewBoard(BOARD_X, BOARD_Y)
        b.move(0, 0, RIGHT)
        b.move(1, 0, TOP)
        b.move(1, 0, RIGHT)
        b.move(1, 0, BOTTOM)
        try:
            b.move(1, 0, LEFT)
        except ValueError:
            pass

    def test_win(self):
        b = NewBoard(BOARD_X, BOARD_Y)
        self.assertEqual(b.move(0, 0, RIGHT), [])
        self.assertEqual(b.move(0, 0, TOP), [])
        self.assertEqual(b.move(0, 0, BOTTOM), [])
        self.assertEqual(b.move(0, 0, LEFT), [(0,0)])

        b = NewBoard(BOARD_X, BOARD_Y)
        print(b.lines)
        self.assertEqual(b.move(0, 0, LEFT), [])
        print(b.lines)
        self.assertEqual(b.move(0, 0, TOP), [])
        print(b.lines)
        self.assertEqual(b.move(0, 0, BOTTOM), [])
        print(b.lines)
        
        self.assertEqual(b.move(1, 0, RIGHT), [])
        print(b.lines)
        self.assertEqual(b.move(1, 0, TOP), [])
        self.assertEqual(b.move(1, 0, BOTTOM), [])
        
        self.assertEqual(b.move(1, 0, LEFT).sort(), [(0,0),(0,1)].sort())


unittest.main(argv=[''], verbosity=2, exit=False)

test_drawline_dup (__main__.TestBoard) ... ok
test_drawline_top_bottom_conflict (__main__.TestBoard) ... ok
test_mapping1 (__main__.TestBoard) ... ok
test_win (__main__.TestBoard) ... 

[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]]
[[2, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]]
[[3, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]]
[[7, 0, 0, 0], [1, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]]
[[7, 8, 0, 0], [1, 0, 0, 0], [2, 0, 0, 0], [0, 0, 0, 0]]


ok

----------------------------------------------------------------------
Ran 4 tests in 0.014s

OK
